In [11]:
from transformers import AutoConfig, EvalPrediction, AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from datasets import load_dataset, Dataset
from evaluate import load
import numpy as np
# import wandb
import os
import sys
import pandas as pd
import torch
import torch.nn as nn

In [12]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_with_head = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
pre_trained_model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

Insert a layer in the middle

In [34]:
def create_classifier_layer(layer_to_insert):
    till_layer_to_insert = pre_trained_model.encoder.layer[:layer_to_insert]
    after_layer_to_insert = pre_trained_model.encoder.layer[layer_to_insert:]
    model_with_middle_classifier = till_layer_to_insert + [nn.Linear(768, 2)] + after_layer_to_insert

    return model_with_middle_classifier

In [36]:
model_with_middle_classifier = create_classifier_layer(layer_to_insert=4)
model_with_middle_classifier

ModuleList(
  (0-3): 4 x BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (4): Lin

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

Test with the data

In [45]:
file_path = 'data/acronym_data.txt'
data = []

with open(file_path, "r", errors='ignore') as file:
    for line in file.readlines():
        split = line.strip().split('|')
        
        # build the sentence structure
        source_sentence = split[6]
        compare_sentence = source_sentence[:int(split[3])] + split[1] + source_sentence[int(split[4]):]

        row = {
            'source_sentence': source_sentence,
            'compare_sentence': compare_sentence,
            'label': 1
        }
        data.append(row)

data_dict = {key: [item[key] for item in data] for key in data[0]}
dataset = Dataset.from_dict(data_dict)


In [53]:
tokens = tokenizer(dataset[0]['source_sentence'], dataset[0]['compare_sentence'], return_tensors='pt')

In [62]:
output = model_with_head(**tokens)
torch.argmax(output.logits)

tensor(1)

In [66]:
output = model_with_middle_classifier(**tokens)

TypeError: _forward_unimplemented() got an unexpected keyword argument 'input_ids'